In [2]:
import zipfile
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('D:/DS Projects/dsProject/joints/JOINTS-UGM/Datasets/train.csv.zip',compression='zip')
df.tail()

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_9528\1630652430.py:1: DtypeWarning: Columns (1,3,5,6,7,8,9,10,11,12,13,16,17,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('D:/DS Projects/dsProject/joints/JOINTS-UGM/Datasets/train.csv.zip',compression='zip')


,Unnamed: 0,floors_before_eq (total),old_building,plinth_area (ft^2),height_before_eq (ft),land_surface_condition,type_of_foundation,type_of_roof,type_of_ground_floor,type_of_other_floor,...,type_of_reinforcement_concrete,residential_type,no_family_residing,public_place_type,industrial_use_type,govermental_use_type,flexible_superstructure,wall_binding,wall_material,damage_grade
722810,328930,floor two,32.0,NaN,12.0,Steep slope,Clay Sand Mixed mortar-Stone/Brick,NaN,Clay,Wood-Mud or Bamboo Mud,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,1.0,0.0,4.0
722811,246669,three Story,45.0,NaN,18.0,Moderate slope,Clay mortar-Stone/Brick,NaN,Mud,Lumber-plank,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
722812,308764,Three,72.0,NaN,21.0,Flat,Clay Sand Mixed mortar-Stone/Brick,NaN,Clay,TImber/Bamboo-Mud,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,1.0,5.0
722813,79920,one,22.0,NaN,6.0,Flat,Clay Sand Mixed mortar-Stone/Brick,NaN,Clay,Not applicable,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
722814,37198,Floor 3,32.0,NaN,18.0,Flat,Clay Sand Mixed mortar-Stone/Brick,NaN,Brick or Stone,TImber/Bamboo-Mud,...,0.0,Non-residential,None,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0


In [43]:
# Count the number of missing values in each row
missing_values_count = df.isnull().sum(axis=1)

# Filter rows
less_miss = df[missing_values_count <= 1]

In [44]:
less_miss

,Unnamed: 0,floors_before_eq (total),old_building,plinth_area (ft^2),height_before_eq (ft),land_surface_condition,type_of_foundation,type_of_roof,type_of_ground_floor,type_of_other_floor,...,type_of_reinforcement_concrete,residential_type,no_family_residing,public_place_type,industrial_use_type,govermental_use_type,flexible_superstructure,wall_binding,wall_material,damage_grade
0,0,2,1.0,256 ft^2,22.0,Flat,Bamboo or Timber,Bamboo/Timber Light roof,Clay,TImber/Bamboo-Mud,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,0.0,0.0,1.0
1,1,3,3.0,985 ft^2,18.0,Flat,Clay Sand Mixed mortar-Stone/Brick,Wood Light Roof or Bamboo Heavy Roof,Clay,TImber/Bamboo-Mud,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
3,3,2,18.0,185 ft^2,15.0,Flat,Clay Sand Mixed mortar-Stone/Brick,Wood Light Roof or Bamboo Light Roof,Clay,TImber/Bamboo-Mud,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,4.0
4,4,2,22.0,290 ft^2,17.0,Flat,Clay Sand Mixed mortar-Stone/Brick,Bamboo or Timber Light roof,Clay,Timber Mud or Bamboo-Mud,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,1.0
7,7,2,40.0,504 ft^2,14.0,Flat,Clay mortar-Stone/Brick,Bamboo/Timber Light roof,Clay,TImber/Bamboo-Mud,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520004,520004,1,30.0,276 ft^2,9.0,Moderate slope,Clay Sand Mixed mortar-Stone/Brick,Wood Light Roof or Bamboo Light Roof,Clay,Not applicable,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,4.0
520006,520006,2,8.0,432 ft^2,14.0,Flat,Bamboo/Timber,Wood Light Roof or Bamboo Heavy Roof,Clay,Wood-Plank,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,0.0,0.0,2.0
520007,520007,2,26.0,392 ft^2,19.0,Flat,Clay Sand Mixed mortar-Stone/Brick,Bamboo/TImber-Light Roof,Clay,TImber/Bamboo-Mud,...,0.0,Non-residential,None,Non-public,Infrastructure,Non-govermental,unavailable,5.0,2.0,3.0
520009,520009,2,17.0,300 ft^2,16.0,Flat,Clay mortar-Stone/Brick,Wood Light Roof or Bamboo Light Roof,Clay,Wood-Mud or Bamboo Mud,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,available,5.0,2.0,5.0


In [46]:
# Function to calculate local mean
def local_mean_imputation(series, window=2):
    # Create a copy of the series to avoid modifying the original data
    series_copy = series.copy()
    for i in range(len(series)):
        # Check if the value is missing
        if pd.isna(series_copy.iloc[i]):
            # Calculate the local mean for the missing value
            start = max(0, i - window)
            end = min(len(series), i + window + 1)
            local_mean = series_copy.iloc[start:end].mean()
            # Replace the missing value with the local mean
            series_copy.iloc[i] = local_mean
    return series_copy

# Apply local mean imputation to each column in the DataFrame
window_size = 2
for column in less_miss.columns:
    less_miss[column] = local_mean_imputation(less_miss[column], window=window_size)

# Print the DataFramless_missafter local mean imputation
print(less_miss)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_9528\155797934.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  less_miss[column] = local_mean_imputation(less_miss[column], window=window_size)


TypeError: unsupported operand type(s) for +: 'int' and 'str'